In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA


In [2]:
# read pdfs
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [3]:
len(final_documents)

316

In [31]:
# Embedding using huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",      #sentence-transformers/all-MiniLM-l6-v2 //BAAI/bge-small-en-v1.5
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

c:\Users\Sheela Sai kumar\Documents\UPSkilling\Simple-Chatbot-using-Langchain\explore_lang\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [33]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [34]:

retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000174AC143190> search_kwargs={'k': 3}


In [36]:
from dotenv import load_dotenv
load_dotenv()
import os
huggingface_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [48]:

from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(

    repo_id="mistralai/Mistral-Nemo-Instruct-2407",
    model_kwargs={"temperature":0.1,"max_length":500},
    huggingfacehub_api_token= huggingface_api_token

)
query="What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage? What is the health insurance coverage? Health insurance coverage is a type of insurance that covers the cost of medical and surgical expenses incurred by the insured. Here are some key aspects of health insurance coverage:\n\n1. **Types of Coverage:**\n   - **Indemnity Plans (Fee-for-Service):** These plans allow you to visit any healthcare provider, and the insurance company will reimburse you for a portion of the cost.\n   - **Managed Care Plans:**\n     - **Health'

In [52]:
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_api_token
# from langchain_community.llms import HuggingFacePipeline
# hf = HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-Nemo-Instruct-2407",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 10},
# )
# llm = hf 
# llm.invoke(query)

In [40]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [41]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [53]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [54]:

query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [55]:
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

8 U.S. Census Bureau
which was the state’s uninsured 
rate in 2019, the year before it 
expanded Medicaid eligibility.22
Declines in the uninsured rate in 
the majority of these states were 
related to changes in their public 
and private coverage rates. For 
seven of the states with lower 
uninsured rates in 2022, the dif -
ference was driven by increases 
in private coverage. These states 
were Florida, Kansas, Mississippi, 
North Carolina, Ohio, South 
Carolina, and Texas.
For seven states, the uninsured 
rate decrease was related to 
increases in public coverage with 
no corresponding change in the 
level of private coverage. These 
states were Alabama, California, 
Georgia, Illinois, Indiana, Michigan, 
and Oklahoma. In three states 
(Missouri, New York, and Virginia), 
it was shifts in coverage from pri -
vate to public that contributed to 
the decline in 